# Part 2: Advanced Retrieval Strategies

In Part 1, we built a "naive" RAG pipeline. It works, but it has flaws:
1. **Missed Context**: Splitting documents arbitrarily might cut a key idea in half.
2. **Bad Access Patterns**: Searching for a specific keyword might miss a document that uses a synonym.
3. **Distraction**: Retrieving 10 documents might confuse the LLM if only 1 is relevant ("Lost in the Middle" phenomenon).

In this notebook, we fix these issues.

In [12]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader

load_dotenv()
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://VAS_RESOURCE_NAME.openai.azure.com/")
api_key = os.getenv("AZURE_OPENAI_API_KEY", "VAS_API_KEY") # V prostředí často jako AZURE_OPENAI_API_KEY
deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002") # Název nasazení modelu (ne modelu samotného!)
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15") 

# Inicializace Azure Embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    azure_deployment=deployment_name,
    openai_api_version=api_version,
)

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # Předpokládáme, že název deploymentu v Azure je "gpt-4o"
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0
)

## 1. MultiQuery Retriever (Inteligentní rozšíření dotazu)

**Představte si to jako**: Když něco hledáte na Googlu a na první pokus nenajdete to, co chcete, zkusíte dotaz přeformulovat. Zkusíte synonyma, odbornější výrazy nebo se zeptáte z jiného úhlu. **MultiQuery Retriever** dělá přesně tohle za vás – automaticky a okamžitě.

### 🕵️‍♂️ V čem je problém? (Distance-based vector search)
Vektorové databáze hledají dokumenty podle "významové vzdálenosti". Někdy se ale stane, že **slova uživatele** a **slova v dokumentu** jsou příliš odlišná, i když popisují stejnou věc.
*   *Uživatel napíše:* "Nefunguje účtování."
*   *Dokument obsahuje:* "Protokol pro řešení transakčních výjimek."

Pokud položíte jen jednu otázku, máte jen **jeden pokus** trefit se do správného místa v databázi. Pokud se netrefíte, RAG selže.

### 💡 Řešení: "Rozhození širší sítě"
Místo toho, abychom poslali do databáze jen váš jeden nedokonalý dotaz, využijeme pomoc LLM:

1.  **Generování variant**: LLM si vezme vaši otázku a vymyslí 3–5 různých způsobů, jak se zeptat na to samé (použije synonyma, různé perspektivy).
2.  **Hromadné hledání**: Spustí se vyhledávání pro **KAŽDOU** z těchto variant zvlášť.
3.  **Sjednocení (Unique Union)**: Systém posbírá všechny nalezené dokumenty ze všech dotazů, odstraní duplicity (ty, které se našly vícekrát) a vrátí vám kompletní sadu.

**Příklad z praxe:**
*   **Váš dotaz:** *"How to handle server outages?"*
*   **Co udělá MultiQuery (na pozadí):** Vygeneruje a hledá také:
    1.  *"What are the protocols for system downtime?"*
    2.  *"Emergency procedures for server failure"*
    3.  *"Recovery steps during network incidents"*

Díky tomu "prohledáte" mnohem větší část databáze a máte jistotu, že vám neunikne klíčový dokument jen kvůli špatně zvolenému slovíčku.

In [28]:
# 1. Inicializace (jako doposud)
# Ujistěte se, že vectorstore je správně načtený s persist_directory="../chroma_db"
try:
    from langchain.retrievers.multi_query import MultiQueryRetriever
except ImportError:
    from langchain_classic.retrievers import MultiQueryRetriever

retriever_mq = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

question = "How to handle server outages?"

# --- FUNKČNÍ ANALÝZA ---
print(f"❓ Původní dotaz: '{question}'")
print("\n--- 1. Fáze: Generování variant (Explicitní spuštění) ---")

# Zde si "vytáhneme" vnitřní řetězec, který má na starosti pouze generování
# a spustíme ho samostatně, abychom viděli výsledek.
if hasattr(retriever_mq, "llm_chain"):
    # Spuštění generování
    variants_result = retriever_mq.llm_chain.invoke({"question": question})
    
    # Výsledek může být objekt nebo slovník, vytiskneme text
    if isinstance(variants_result, dict) and "text" in variants_result:
        print(variants_result["text"])
    else:
        print(variants_result)
else:
    print("❌ Nepodařilo se nalézt atribut 'llm_chain'.")

print("\n--- 2. Fáze: Samotné vyhledávání dokumentů ---")
# Nyní proběhne standardní proces (znovu si vygeneruje otázky a vyhledá je)
docs = retriever_mq.invoke(question)

print(f"✅ Nalezeno {len(docs)} unikátních dokumentů.")
for i, d in enumerate(docs):
    print(f"   [{i+1}] {d.page_content[:80]}... (Zdroj: {d.metadata.get('source', 'N/A')})")

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

print("\n--- 3. Fáze: Generování finální odpovědi (LCEL RAG Chain) ---")

# A. Prompt (Instrukce pro model)
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# B. Pomocná funkce pro zformátování dokumentů do textu
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# C. Sestavení RAG řetězce (Chain)
# ZMĚNA: Místo 'retriever' použijeme náš 'retriever_mq'
rag_chain_from_multiquery = (
    {"context": retriever_mq | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# D. Spuštění
# .invoke() spustí celý proces: MultiQuery vyhledávání (s 3-5 variantami) -> formátování -> prompt -> GPT
print(f"Otázka: {question}")
print("-" * 30)
response = rag_chain_from_multiquery.invoke(question)
print("Odpověď:", response)

❓ Původní dotaz: 'How to handle server outages?'

--- 1. Fáze: Generování variant (Explicitní spuštění) ---
['What are the best practices for managing server downtime?', 'How can I effectively deal with server outages?', 'What strategies can be used to handle server failures?']

--- 2. Fáze: Samotné vyhledávání dokumentů ---
✅ Nalezeno 4 unikátních dokumentů.
   [1] SERVICE LEVEL AGREEMENT (SLA) for NebulaDB Enterprise

1. DEFINITIONS
"Uptime" r... (Zdroj: ../data/legal/sla_contract.txt)
   [2] 4. EXCLUSIONS
The Service Commitment does not apply to any unavailability, suspe... (Zdroj: ../data/legal/sla_contract.txt)
   [3] Quarter: Q2 2024
Revenue: 1200000
Expenses: 850000
Profit: 350000
Notes: cost op... (Zdroj: ../data/finance/financial_report.csv)
   [4] 3. SERVICE CREDITS
If we do not meet the Service Commitment, you will be eligibl... (Zdroj: ../data/legal/sla_contract.txt)

--- 3. Fáze: Generování finální odpovědi (LCEL RAG Chain) ---
Otázka: How to handle server outages?
-------

## 2. Parent Document Retriever (Malý detektiv, Velký čtenář)

Abychom pochopili, proč tuto techniku potřebujeme, musíme si nejdříve ukázat, kde **klasický RAG selhává**.

### 1. Klasický přístup (Bez Parent Retrieveru)
V klasickém RAGu děláte kompromis. Musíte se rozhodnout pro jednu velikost kousků (chunks) pro všechno.
*   Jeden chunk slouží k tomu, aby byl **nalezen** (vyhledávání).
*   TEN SAMÝ chunk se pošle LLM, aby z něj **odpovědělo** (generování).

**Dilema:**
*   **Když uděláte chunks malé (např. 1 věta):**
    *   ✅ **Vyhledávání je super:** Když uživatel hledá *"cena podpory"*, počítač to snadno najde, protože ve větě *"Cena podpory je 500 Kč"* tvoří tato slova 50 % obsahu. Je to jasný signál.
    *   ❌ **LLM selže:** Modelu pošlete jen tu jednu větu: *"Cena podpory je 500 Kč"*. LLM se zeptá: *"Aha, a jaké podpory? Měsíční? Roční? Pro koho?"* Chybí mu **kontext**.

*   **Když uděláte chunks velké (např. celá stránka):**
    *   ✅ **LLM je spokojené:** Vidí celou stránku, ví, že jde o *"NebulaDB Enterprise roční plán"*. Má kontext.
    *   ❌ **Vyhledávání selže:** Ve velké stránce textu se slova *"cena podpory"* ztratí. Tvoří třeba jen 1 % textu. Vektorová podobnost bude nízká a databáze tento dokument vůbec nenajde (tzv. "utopení jehly v kupce sena").

### 2. Řešení: Parent Document Retriever
Tato metoda **odděluje** to, co hledáme, od toho, co čte LLM.

1.  **Indexujeme "Děti" (Small Chunks):** Dokument rozsekáme na maličké kousky (např. 100 znaků). Ty použijeme **POUZE pro vyhledávání**. Díky tomu je hledání extrémně citlivé a přesné.
2.  **Ukládáme "Rodiče" (Large Chunks):** Zároveň si pamatujeme, že tyto malé kousky patří do většího bloku (např. 500 znaků nebo celá stránka).
3.  **Proces:**
    *   Uživatel hledá *"cena"*.
    *   Databáze najde malý kousek (Dítě): *"Cena je 500 Kč"*.
    *   Retriever ale nevrátí toto dítě. Podívá se, kdo je jeho rodič.
    *   LLM pošle celého rodiče: *"Ceník pro rok 2024. Služba NebulaDB. Základní cena je zdarma. Cena prémiové podpory je 500 Kč měsíčně při úvazku na rok."*

### 3. Proč "LLM funguje lépe s velkými kousky"?
Ptáte se správně: *"LLM se používá vždy, ne?"*
Ano, ale LLM je jako **kuchař**. Jeho jídlo (odpověď) je jen tak dobré, jak dobré jsou **suroviny** (kontext), které mu dodáte.

*   **Příklad:** Uživatel se zeptá: *"Co se stane při výpadku?"*
*   **Vstup pro LLM (bez Parent Retrieveru - malý chunk):**
    > *"Dostanete kredit 10 %."*
    *   *Výsledek:* LLM odpoví: "Dostanete 10% kredit." (Neví za co, neví komu).
*   **Vstup pro LLM (s Parent Retrieverem - velký chunk):**
    > *"3. KOMPENZACE. Pokud dostupnost klesne pod 99.9%, zákazník má nárok na kompenzaci. U výpadku nad 1 hodinu dostanete kredit 10 % z měsíční platby."*
    *   *Výsledek:* LLM odpoví: "Pokud výpadek trvá déle než hodinu a dostupnost klesne pod 99.9 %, máte nárok na kredit ve výši 10 % z měsíční platby."

**Závěr:** Parent Document Retriever nám dává to nejlepší z obou světů: **Přesnost vyhledávání** v malých datech a **bohatost kontextu** ve velkých datech.

In [37]:
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_classic.storage import InMemoryStore
# Používáme moderní cestu pro splittery
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# 1. Příprava úložišť
# Vectorstore (Chroma) bude držet jen "malé děti" (vektory)
# Docstore (InMemory) bude držet "velké rodiče" (celý text)
vectorstore_parent = Chroma(
    collection_name="rag_parent", 
    embedding_function=embeddings,
    # Tady nepoužíváme persistenci na disk pro zjednodušení ukázky, 
    # ale v praxi byste chtěli i 'store' ukládat (např. RedisStore).
)
store = InMemoryStore()

# 2. Nastavení Splitterů (Dělení textu)
# "Rodič" - větší bloky (např. 500 znaků), které chce číst LLM
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
# "Dítě" - malé útržky (např. 100 znaků), které slouží jen pro vyhledání
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=50)

# 3. Inicializace Retrieveru
retriever_parent = ParentDocumentRetriever(
    vectorstore=vectorstore_parent, 
    docstore=store, 
    child_splitter=child_splitter,  # Podle tohoto se hledá
    parent_splitter=parent_splitter # Toto se vrací do promptu
)

# 4. Načtení dat a indexace
print("--- Indexuji dokumenty ---")
# Načteme náš Markdown soubor
loader = TextLoader("../data/tech_docs/api_docs.md")
docs = loader.load()

# Tady se děje kouzlo: add_documents samo rozseká text na malé i velké kusy a propojí je
retriever_parent.add_documents(docs)
print(f"Indexováno {len(docs)} původní dokumentů.")

# 5. Test vyhledávání
query = "rate limits"
print(f"\n❓ Hledám dotaz: '{query}'")

# Použijeme .invoke()
results = retriever_parent.invoke(query)

if len(results) > 0:
    print(f"✅ Nalezeno dokumentů: {len(results)}")
    print(f"   Délka vráceného obsahu: {len(results[0].page_content)} znaků")
    print(f"   Ukázka obsahu (Rodič):")
    print("-" * 20)
    print(results[0].page_content)
    print("-" * 20)
else:
    print("❌ Nic nenalezeno.")

--- Indexuji dokumenty ---
Indexováno 1 původní dokumentů.

❓ Hledám dotaz: 'rate limits'
✅ Nalezeno dokumentů: 1
   Délka vráceného obsahu: 143 znaků
   Ukázka obsahu (Rodič):
--------------------
## Error Codes
- `400`: Invalid parameters (e.g., node_count < 3).
- `401`: unauthorized.
- `429`: Rate limit exceeded (100 req/sec per token).
--------------------


## 3. Contextual Compression (Destilace informací)

Tato technika řeší problém **"informačního šumu"**.

### 1. Problém: Vektorové hledání je "hloupé"
Vektorová databáze je velmi rychlá, ale neumí číst. Porovnává pouze matematickou podobnost vektorů.
*   **Příklad:** Hledáte *"Jakou dostanu slevu za výpadek?"*
*   **Vector Store najde:** *"Kompenzace (slevy) se nevztahují na plánované výpadky údržby..."*
    *   Matematicky je to velmi podobná věta (obsahuje slova *výpadek, sleva*).
    *   Fakticky je to ale **opak** toho, co chcete (říká, kdy slevu nedostanete).
*   **Výsledek:** Pokud pošlete LLM 10 takových dokumentů, zahltíte ho balastem (tzv. "lost in the middle").

### 2. Řešení: Contextual Compression (Re-ranking)
Představte si to jako dvoufázový proces **rýžování zlata**:

1.  **Bagr (Base Retriever):** Nejdříve nabereme velkou lžíci hlíny a kamení. Řekneme databázi: *"Dej mi všechno, co aspoň trochu souvisí s dotazem."* (Např. 20 dokumentů).
2.  **Pinzeta (Compressor/Nové LLM):** Poté vezmeme chytrý model (LLM), který si těch 20 dokumentů přečte a řekne: *"Tohle je jen o plánované údržbě - vyhodit. Tohle je o slevách na rohlíky - vyhodit. Tady! Tohle je o slevách za výpadek - nechat."*

Navíc model dokáže text **"vyždímat"** (komprimovat). Pokud je dokument dlouhý 500 slov, ale odpověď je v jedné větě, model všechno ostatní smaže a vrátí vám jen tu jednu zlatou větu.

In [ ]:
# Importy - snažíme se načíst ze správného místa pro vaši verzi
try:
    from langchain.retrievers import ContextualCompressionRetriever
    from langchain.retrievers.document_compressors import LLMChainExtractor
except ImportError:
    from langchain_classic.retrievers import ContextualCompressionRetriever
    from langchain_classic.retrievers.document_compressors import LLMChainExtractor

# 1. Vytvoření kompresoru
# Tento "inteligentní filtr" používá LLM k posouzení každého dokumentu
compressor = LLMChainExtractor.from_llm(llm)

# 2. Sestavení Retrieveru
# Spojíme "Bagr" (vectorstore -- hledá hrubě) a "Pinzetu" (compressor -- čistí)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 5}) # Vytáhni z databáze 5 nejlepších položek, které najdeš. V kontextu RAG jsou tyto "položky" technicky vzato Chunky (ty rozsekané kousky textu, které jsme vytvořili pomocí splitteru). Pro LangChain je to objekt Document, ale fakticky je to Chunk.
)

# 3. Testování
query = "What is the SLA credit for 96% uptime?"
print(f"❓ Dotaz: '{query}'\n")

# Zkusíme najít dokumenty
compressed_docs = compression_retriever.invoke(query)

print(f"✅ Nalezeno relevantních úryvků: {len(compressed_docs)}")
if len(compressed_docs) > 0:
    print("-" * 30)
    # Všimněte si, že vrácený text je kratší než originál - je "vyčištěný"
    print(compressed_docs[0].page_content)
    print("-" * 30)
    print(f"(Zdroj: {compressed_docs[0].metadata.get('source', 'neznámý')})")
else:
    print("❌ Model vyhodnotil, že žádný dokument neobsahuje přímou odpověď.")

❓ Dotaz: 'What is the SLA credit for 96% uptime?'

✅ Nalezeno relevantních úryvků: 1
------------------------------
- < 99.0% but >= 95.0%: 25% Credit
------------------------------
(Zdroj: ../data/legal/sla_contract.txt)


## Conclusion
We have explored:
- **MultiQuery**: For vague user intent.
- **ParentDocument**: For better context management.
- **Compression**: For precision filtering.

In the next notebook, we will learn how to **evaluate** which of these is actually better for our use case using RAGAS.